In [0]:
%tensorflow_version 2.x

import tensorflow as tf 
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model,load_model,Sequential
from keras.layers import GlobalAveragePooling2D,Dense,Conv2D,Flatten,Dropout
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau,EarlyStopping 
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.applications.mobilenet import MobileNet
import os 
import cv2
from keras.applications.densenet import DenseNet121



Using TensorFlow backend.


In [0]:
from google.colab import files
files.upload()

In [0]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6
!kaggle -v
!mkdir -p ~/.kaggle 
!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 2.8MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 1.7 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=a75dd5dfb65a577a32b849fa61c7cad4b375174a2357f837d30feebbc0e6e72b
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.6/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.6/dist-packages/kaggle/api/kaggle_api_extended.py", line 149, in authenticate
    self.config_file, self.config_dir))
OSError: Could not find kaggle.json. Make sure

In [0]:
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

 98% 794M/814M [00:15<00:00, 34.6MB/s]
100% 814M/814M [00:15<00:00, 54.3MB/s]


In [0]:

from zipfile import ZipFile
file_name="dogs-vs-cats-redux-kernels-edition.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done


In [0]:
from zipfile import ZipFile
file_name="train.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

from zipfile import ZipFile
file_name="test.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

Done
Done


In [0]:
gcs_path='gs://kds-de539a08dcc2de245bd59dfeeba193e0d26abeecdf0286697120c16e'
list1=os.listdir("/content/train")
list2=os.listdir("/content/test")

In [0]:
list3=[]
for i in range(len(list1)):
  if list1[i][0:3]=='cat':
    list3.append('cat')
  else:
    list3.append('dog')

In [0]:
df_train=pd.DataFrame({'filenames':list1,'type':list3})
df_test=pd.DataFrame({'filenames':list2})

In [0]:
df_train.head(),df_test.head()

(       filenames type
 0   dog.2970.jpg  dog
 1   dog.1355.jpg  dog
 2  dog.10252.jpg  dog
 3   cat.5259.jpg  cat
 4   dog.4379.jpg  dog,    filenames
 0   4890.jpg
 1  10765.jpg
 2  11617.jpg
 3   9204.jpg
 4   1074.jpg)

In [0]:
train_imagedatagenerator=ImageDataGenerator(
     horizontal_flip=True,rescale=1/255,validation_split=0.1
                            
)

test_imagedatagenerator=ImageDataGenerator(
    rescale=1/255
)
"""rotation_range=10,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            zoom_range=.1,
                            fill_mode='nearest',
                            shear_range=0.1,
                            
                            brightness_range=[0.5, 1.5],
                            """


"rotation_range=10,\n                            width_shift_range=0.1,\n                            height_shift_range=0.1,\n                            zoom_range=.1,\n                            fill_mode='nearest',\n                            shear_range=0.1,\n                            \n                            brightness_range=[0.5, 1.5],\n                            "

In [0]:
train=train_imagedatagenerator.flow_from_dataframe(
    df_train,
    directory='/content/train',
    x_col='filenames',
    y_col='type',
    class_mode='categorical',
    batch_size=64,
    shuffle=True,
    target_size=(224,224),
    subset='training',
)
val=train_imagedatagenerator.flow_from_dataframe(
    df_train,
    directory='/content/train',
    x_col='filenames',
    y_col='type',
    class_mode='categorical',
    batch_size=64,
    shuffle=True,
    target_size=(224,224),
    subset='validation',
)
test=test_imagedatagenerator.flow_from_dataframe(
   df_test,
    directory='/content/test',
    x_col='filenames',
    y_col=None,
    class_mode=None,
    batch_size=64,
    shuffle=False,
    target_size=(224,224)

)

Found 22500 validated image filenames belonging to 2 classes.
Found 2500 validated image filenames belonging to 2 classes.
Found 12500 validated image filenames.


In [0]:
mobile_net=MobileNet(include_top=False,weights='imagenet',input_shape=(224,224,3))
x=mobile_net.output
x=Flatten()(x)
x=Dense(128,activation='relu')(x)
x=Dropout(0.2)(x)
x=Dense(64,activation='relu')(x)
x=Dropout(0.1)(x)
predictions=Dense(2,activation='softmax')(x)

model1=Model(inputs=mobile_net.input,outputs=predictions)

for layer in mobile_net.layers:
  layer.trainable=False

model1.summary()

Model: "model_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128

In [0]:
layers_info=[(layer,layer.name,layer.trainable) for layer in model1.layers]

layer_info=pd.DataFrame(layers_info,columns=['layer_type','layer_name','trainable'])
layer_info.tail(25)

,layer_type,layer_name,trainable
68,<keras.layers.convolutional.DepthwiseConv2D ob...,conv_dw_11,False
69,<keras.layers.normalization.BatchNormalization...,conv_dw_11_bn,False
70,<keras.layers.advanced_activations.ReLU object...,conv_dw_11_relu,False
71,<keras.layers.convolutional.Conv2D object at 0...,conv_pw_11,False
72,<keras.layers.normalization.BatchNormalization...,conv_pw_11_bn,False
73,<keras.layers.advanced_activations.ReLU object...,conv_pw_11_relu,False
74,<keras.layers.convolutional.ZeroPadding2D obje...,conv_pad_12,False
75,<keras.layers.convolutional.DepthwiseConv2D ob...,conv_dw_12,False
76,<keras.layers.normalization.BatchNormalization...,conv_dw_12_bn,False
77,<keras.layers.advanced_activations.ReLU object...,conv_dw_12_relu,False


In [0]:
model1.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [0]:
model_checkpoint=ModelCheckpoint('best_mobile_net.h5',monitor='val_accuracy',save_best_only=True,mode='max')
early_stopping=EarlyStopping(monitor="val_accuracy",min_delta=0,patience=10,verbose=0,mode="max",restore_best_weights=True)
reduce_lr=ReduceLROnPlateau(monitor="loss",factor=0.2,patience=10,min_lr=0.001)

In [0]:
history1=model1.fit_generator(train,epochs=15,steps_per_epoch=train.n//train.batch_size,validation_data=val,verbose=1,callbacks=[model_checkpoint,early_stopping,reduce_lr])

Epoch 1/15
351/351 [==============================] - 91s 259ms/step - loss: 2.0467 - accuracy: 0.9333 - val_loss: 2.5051e-04 - val_accuracy: 0.9808
Epoch 2/15
351/351 [==============================] - 90s 255ms/step - loss: 0.0728 - accuracy: 0.9744 - val_loss: 8.9844e-05 - val_accuracy: 0.9768
Epoch 3/15
351/351 [==============================] - 89s 254ms/step - loss: 0.0791 - accuracy: 0.9745 - val_loss: 5.9604e-06 - val_accuracy: 0.9804
Epoch 4/15
351/351 [==============================] - 90s 255ms/step - loss: 0.0654 - accuracy: 0.9756 - val_loss: 2.0266e-06 - val_accuracy: 0.9824
Epoch 5/15
351/351 [==============================] - 90s 256ms/step - loss: 0.0561 - accuracy: 0.9796 - val_loss: 0.0030 - val_accuracy: 0.9812
Epoch 6/15
 21/351 [>.............................] - ETA: 1:24 - loss: 0.0382 - accuracy: 0.9844

KeyboardInterrupt: ignored

In [0]:
testing_model=load_model('/content/best_mobile_net.h5')

In [0]:
test.reset()

In [0]:
sub_pred=model1.predict_generator(test,verbose=1)

196/196 [==============================] - 44s 225ms/step


In [0]:
sample_submission=pd.read_csv('/content/sample_submission.csv')

In [0]:
sub_pred[0]

array([1.0576932e-04, 9.9989426e-01], dtype=float32)

In [0]:
dogs_pred=[]
for i in range(len(sub_pred)):
  dogs_pred.append(sub_pred[i][1])

In [0]:
submission=pd.DataFrame({'id':sample_submission['id'],'label':dogs_pred})

In [0]:
submission.to_csv('submission.csv',index=False)

In [0]:
#dogs_pred=np.clip(dogs_pred,a_min=0.05,a_max=0.95)

In [0]:
#submission=pd.DataFrame({'id':sample_submission['id'],'label':dogs_pred})
#submission.to_csv('submission.csv',index=False)


In [0]:
#dogs_pred

In [0]:
from keras.preprocessing import image
imagePath='/content/train/cat.5259.jpg'
test_image2 = image.load_img(imagePath, target_size = (224, 224)) 
test_image2 = image.img_to_array(test_image2)
test_image2 = np.expand_dims(test_image2, axis = 0)

#predict the result
result = model1.predict(test_image2)

In [0]:
result

array([[0.89309716, 0.10690282]], dtype=float32)

In [0]:
list1

['dog.2970.jpg',
 'dog.1355.jpg',
 'dog.10252.jpg',
 'cat.5259.jpg',
 'dog.4379.jpg',
 'cat.2114.jpg',
 'dog.5173.jpg',
 'cat.9115.jpg',
 'dog.10038.jpg',
 'dog.2374.jpg',
 'dog.2892.jpg',
 'cat.7547.jpg',
 'dog.11154.jpg',
 'cat.9446.jpg',
 'cat.1761.jpg',
 'dog.4796.jpg',
 'cat.8397.jpg',
 'dog.3562.jpg',
 'cat.9777.jpg',
 'dog.5728.jpg',
 'dog.949.jpg',
 'dog.10690.jpg',
 'dog.910.jpg',
 'cat.9987.jpg',
 'cat.6614.jpg',
 'cat.9965.jpg',
 'cat.11061.jpg',
 'dog.2604.jpg',
 'dog.8639.jpg',
 'cat.1945.jpg',
 'cat.6558.jpg',
 'cat.9281.jpg',
 'dog.447.jpg',
 'cat.10551.jpg',
 'dog.3080.jpg',
 'cat.12483.jpg',
 'cat.1400.jpg',
 'cat.10983.jpg',
 'dog.7761.jpg',
 'cat.12029.jpg',
 'dog.3158.jpg',
 'dog.1253.jpg',
 'cat.4944.jpg',
 'cat.7453.jpg',
 'dog.2200.jpg',
 'dog.687.jpg',
 'cat.10375.jpg',
 'dog.5950.jpg',
 'cat.4430.jpg',
 'dog.2443.jpg',
 'cat.12254.jpg',
 'cat.769.jpg',
 'cat.3652.jpg',
 'dog.3435.jpg',
 'cat.4705.jpg',
 'cat.10501.jpg',
 'cat.8007.jpg',
 'cat.11228.jpg',
 'dog.